### RAG and Citations using Oracle Vector Store and Cohere Command-R
* show how to use **citations** and have **more grounded answers**

In [1]:
import logging
from pprint import pprint

import oracledb

from langchain_community.vectorstores.oraclevs import OracleVS
from langchain_community.vectorstores.utils import DistanceStrategy
from oci_command_r_oo import OCICommandR

# to compute embeddings vectors
from langchain_community.embeddings import OCIGenAIEmbeddings

from utils import load_configuration

# private information
from config_private import COMPARTMENT_ID, DB_USER, DB_PWD, DB_HOST_IP, DB_SERVICE

#### Settings

In [2]:
# Configure logging
logger = logging.getLogger("ConsoleLogger")

logger.setLevel(logging.INFO)

# load the config in the config.toml file
config = load_configuration()

# embeddings model: we're using OCI GenAI multilingual Cohere
OCI_EMBED_MODEL = config["embeddings"]["oci"]["embed_model"]
EMBED_ENDPOINT = config["embeddings"]["oci"]["embed_endpoint"]

LLM_ENDPOINT = config["llm"]["oci"]["endpoint"]

# number of docs retrieved for each query
# reduced from config to simplify output here
TOP_K = 4

# for AI Vector Search
# to connect to DB
# default port is 1521
DSN = f"{DB_HOST_IP}:1521/{DB_SERVICE}"

print("The complete configuration is:")
print()
pprint(config)

The complete configuration is:

{'embeddings': {'cohere': {},
                'embed_model_type': 'OCI',
                'oci': {'embed_batch_size': 90,
                        'embed_endpoint': 'https://inference.generativeai.us-chicago-1.oci.oraclecloud.com',
                        'embed_model': 'cohere.embed-multilingual-v3.0'}},
 'llm': {'cohere': {'llm_model': 'command-r'},
         'max_tokens': 1024,
         'model_type': 'OCI',
         'oci': {'compartment_ocid': 'ocid1.compartment.oc1..aaaaaaaaushuwb2evpuf7rcpl4r7ugmqoe7ekmaiik3ra3m7gec3d234eknq',
                 'endpoint': 'https://inference.generativeai.us-chicago-1.oci.oraclecloud.com',
                 'llm_model': 'cohere.command-r-16k',
                 'preamble_id': 'preamble3'},
         'temperature': 0.1,
         'top_k': 1,
         'top_p': 0.15},
 'reranker': {'add_reranker': True,
              'cohere_reranker_model': 'rerank-multilingual-v3.0'},
 'retriever': {'top_k': 8, 'top_n': 4},
 'text_splitting':

In [3]:
# utility function
def print_metadata(v_metadata):
    """
    this is the format:
    {'source': './books/oracle-ai-vector-search-users-guide.pdf', 'page': 0}
    """
    print(f"- Source: {v_metadata['source']}, page: {v_metadata['page']}")

In [4]:
# create client for Embeddings and AI Vector Search

# Embed model here is needed to embed the query!
# for embeddings we're using the extension that handles batching
embed_model = OCIGenAIEmbeddings(
    auth_type="API_KEY",
    model_id=OCI_EMBED_MODEL,
    service_endpoint=EMBED_ENDPOINT,
    compartment_id=COMPARTMENT_ID,
)

# create the Vector Store (OracleVS)
#
#
try:
    # we need to provide a connection as input to OracleVS
    connection = oracledb.connect(user=DB_USER, password=DB_PWD, dsn=DSN)
    logger.info("Connection successful!")

    # get an instance of OracleVS
    v_store = OracleVS(
        client=connection,
        table_name="ORACLE_KNOWLEDGE",
        distance_strategy=DistanceStrategy.COSINE,
        embedding_function=embed_model,
    )

except Exception as e:
    logger.error("Connection failed!")
    logger.error(e)

2024-06-07 17:44:48,872 - INFO - Connection successful!


In [5]:
# create a Langchain retriever from the Vector Store
retriever = v_store.as_retriever(search_kwargs={"k": TOP_K})

logger.info("Retriever created...")

2024-06-07 17:44:50,700 - INFO - Retriever created...


#### Set the query

In [6]:
# query = "Could you explain what is JSON Relational Duality in Oracle Database 23c?"
query = "What is the VECTOR type in Oracle Database and how it is used?"

#### The semantic search using AI Vector Search

In [7]:
%%time
# first let's test the semantic search

result_docs = retriever.invoke(query)

# display results
print("")
print("--- Document retrieved from the knowledge base ---")
print()

for i, doc in enumerate(result_docs):
    print("-------------------------------------------")
    print(f"Document n. {i+1}")
    print("")
    print("- Content:")
    print(doc.page_content)
    print("")
    print_metadata(doc.metadata)
    print("-------------------------------------------")
    print("")


--- Document retrieved from the knowledge base ---

-------------------------------------------
Document n. 1

- Content:
4
Store Vector Embeddings
You store the resulting vector embeddings and associated unstructured data with your
relational business data in Oracle Database.
•Create Tables Using the VECTOR Data Type
You can declare a table's column as a VECTOR  data type.
•Insert Vectors in a Database Table Using the INSERT Statement
Once you create a table with a VECTOR  data type column, you can directly insert vectors
into the table using the INSERT  statement.
•Load Vector Data Using SQL*Loader
Use these examples to understand how you can load character and binary vector data.
•Unload and Load Vectors Using Oracle Data Pump
Starting with Oracle Database 23ai, Oracle Data Pump enables you to use multiple
components to load and unload vectors to databases.
Create Tables Using the VECTOR Data Type
You can declare a table's column as a VECTOR  data type.
The following command shows 

#### Get the answer from Cohere Command-R

In [8]:
# for now, no history
chat_history = []

# Cohere wants a map
# take the output from the AI Vector Search
# and trasnform in a format suitable for Cohere command-r
documents_txt = [
    {
        "id": str(i + 1),
        "snippet": doc.page_content,
        "source": doc.metadata["source"],
        "page": str(doc.metadata["page"]),
    }
    for i, doc in enumerate(result_docs)
]

command_r_params = {
    "model": "cohere.command-r-16k",
    "service_endpoint": LLM_ENDPOINT,
    "compartment_id": COMPARTMENT_ID,
    "max_tokens": 1024,
}
# this is a custom class that wraps OCI Python SDK
chat = OCICommandR(**command_r_params)

response = chat.invoke(query=query, chat_history=[], documents=documents_txt)

In [9]:
# have another look at the query
query = "What is the VECTOR type in Oracle Database and how it is used?"

In [10]:
# here we get the text of the answer
print(f"Query: {query}")
print("")
print("Answer:")
print(response.data.chat_response.text)

Query: What is the VECTOR type in Oracle Database and how it is used?

Answer:
The VECTOR data type was introduced with the Oracle Database 23ai release. It allows users to store vector embeddings alongside business data within the database. Vector embeddings can be created from unstructured data using embedding models, enabling semantic queries on the business data.

The VECTOR data type provides flexibility in defining vector columns, allowing an arbitrary number of dimensions and formats. The vectors can be inserted into table columns using the INSERT statement. Various functions are also provided for vector operations, such as fetching vector data, calculating the norm, determining dimensions, and serializing/deserializing vectors.

Here's an example of creating a table with a VECTOR column:
```sql
CREATE TABLE my_vectors (id NUMBER, embedding VECTOR);
```


In [11]:
print(f"Document used to answer:")
print()

for doc in response.data.chat_response.documents:
    print(doc)
    print("")

Document used to answer:

{'id': '1', 'page': '127', 'snippet': "4\nStore Vector Embeddings\nYou store the resulting vector embeddings and associated unstructured data with your\nrelational business data in Oracle Database.\n•Create Tables Using the VECTOR Data Type\nYou can declare a table's column as a VECTOR  data type.\n•Insert Vectors in a Database Table Using the INSERT Statement\nOnce you create a table with a VECTOR  data type column, you can directly insert vectors\ninto the table using the INSERT  statement.\n•Load Vector Data Using SQL*Loader\nUse these examples to understand how you can load character and binary vector data.\n•Unload and Load Vectors Using Oracle Data Pump\nStarting with Oracle Database 23ai, Oracle Data Pump enables you to use multiple\ncomponents to load and unload vectors to databases.\nCreate Tables Using the VECTOR Data Type\nYou can declare a table's column as a VECTOR  data type.\nThe following command shows a simple example:\nCREATE TABLE my_vectors

In [12]:
print(f"Citations:")
print()

for citation in response.data.chat_response.citations:
    print(citation)
    print("")

Citations:

{
  "document_ids": [
    "1",
    "2"
  ],
  "end": 74,
  "start": 45,
  "text": "Oracle Database 23ai release."
}

{
  "document_ids": [
    "1",
    "2"
  ],
  "end": 162,
  "start": 94,
  "text": "store vector embeddings alongside business data within the database."
}

{
  "document_ids": [
    "2"
  ],
  "end": 241,
  "start": 188,
  "text": "created from unstructured data using embedding models"
}

{
  "document_ids": [
    "2"
  ],
  "end": 268,
  "start": 252,
  "text": "semantic queries"
}

{
  "document_ids": [
    "3"
  ],
  "end": 333,
  "start": 313,
  "text": "provides flexibility"
}

{
  "document_ids": [
    "3"
  ],
  "end": 417,
  "start": 362,
  "text": "allowing an arbitrary number of dimensions and formats."
}

{
  "document_ids": [
    "1"
  ],
  "end": 492,
  "start": 437,
  "text": "inserted into table columns using the INSERT statement."
}

{
  "document_ids": [
    "4"
  ],
  "end": 510,
  "start": 501,
  "text": "functions"
}

{
  "document_ids": 

In [13]:
from oci_citations_utils import extract_complete_citations

complete_citations = extract_complete_citations(response)

In [14]:
complete_citations

[{'interval': (45, 74),
  'text': 'Oracle Database 23ai release.',
  'documents': [{'id': '1',
    'source': './books/oracle-ai-vector-search-users-guide.pdf',
    'page': '127'},
   {'id': '2',
    'source': './books/oracle-ai-vector-search-users-guide.pdf',
    'page': '8'}]},
 {'interval': (94, 162),
  'text': 'store vector embeddings alongside business data within the database.',
  'documents': [{'id': '1',
    'source': './books/oracle-ai-vector-search-users-guide.pdf',
    'page': '127'},
   {'id': '2',
    'source': './books/oracle-ai-vector-search-users-guide.pdf',
    'page': '8'}]},
 {'interval': (188, 241),
  'text': 'created from unstructured data using embedding models',
  'documents': [{'id': '2',
    'source': './books/oracle-ai-vector-search-users-guide.pdf',
    'page': '8'}]},
 {'interval': (252, 268),
  'text': 'semantic queries',
  'documents': [{'id': '2',
    'source': './books/oracle-ai-vector-search-users-guide.pdf',
    'page': '8'}]},
 {'interval': (313, 333),